In [1]:
# Model Sara

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score


In [3]:
df = pd.read_csv('/Users/kata/Desktop/GitHub/standby_duty_forecasting/sickness_table.csv', index_col=0)
df['date'] = pd.to_datetime(df['date'])
df['year']= df['date'].dt.year
df['month']= df['date'].dt.month
df['day_of_week'] = df['date'].dt.dayofweek
df['week'] = df['date'].dt.isocalendar().week
df['day_of_month'] = df['date'].dt.day
df['day_of_year'] = df['date'].dt.dayofyear
# change type to not get error
df['week'] = df['week'].astype(np.int64)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1152 entries, 0 to 1151
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          1152 non-null   datetime64[ns]
 1   n_sick        1152 non-null   int64         
 2   calls         1152 non-null   float64       
 3   n_duty        1152 non-null   int64         
 4   n_sby         1152 non-null   int64         
 5   sby_need      1152 non-null   float64       
 6   dafted        1152 non-null   float64       
 7   year          1152 non-null   int64         
 8   month         1152 non-null   int64         
 9   day_of_week   1152 non-null   int64         
 10  week          1152 non-null   int64         
 11  day_of_month  1152 non-null   int64         
 12  day_of_year   1152 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(9)
memory usage: 126.0 KB


In [4]:
# predict calls with RF
# define X and y data
X = df[['year', 'month', 'day_of_week',
       'week', 'n_duty', 'n_sby', 'day_of_month', 'day_of_year']]
y = df.calls

In [5]:
# train test split
def train_test_split(data, n_test):
	return data[:n_test], data[n_test:]


x_train, x_test = train_test_split(X, 922)
y_train, y_test = train_test_split(y, 922)

In [6]:
model = RandomForestRegressor(n_estimators=1000, random_state=6)
model.fit(x_train, y_train)
pred = model.predict(x_test)

In [7]:
# change pred to series for visualizing

pred = pd.Series(pred)
pred = pred.reset_index()
y_test = y_test.reset_index()

In [8]:
# create df to compare data
df_calls_rf = pd.concat([pred, y_test], axis=1).reset_index(drop=True)
df_calls_rf = df_calls_rf.drop(['index'], axis=1)
df_calls_rf['predicted'] = df_calls_rf[0]
df_calls_rf = df_calls_rf.drop([0], axis=1)

In [9]:
# calculate MSE
mse_rf = mean_squared_error(df_calls_rf['calls'], df_calls_rf['predicted'])
print('The MSE for predicting calls with Random Forest is:', mse_rf)

The MSE for predicting calls with Random Forest is: 1526209.779417026


In [10]:
# create df with predicted calls
calls_new = pd.concat([df['calls'].iloc[:922], df_calls_rf['predicted']], axis=0).reset_index(drop=True)
df['calls_new'] = calls_new

In [11]:
# new data for Lasso Regression
# predict calls with RF
# define X and y data
X = df[['year', 'month', 'day_of_week', 'calls_new',
       'week', 'n_duty', 'n_sby', 'day_of_month', 'day_of_year']]
y = df.sby_need

In [12]:
x_train, x_test = train_test_split(X, 922)
y_train, y_test = train_test_split(y, 922)

In [13]:
# Develop Ridge(L2) Regression Model:
# Estimate different values for lamda
alpha = []
cross_val_scores_ridge = []

In [14]:
for i in range(1, 9):
     ridgeModel = Ridge(alpha = i * 0.25)
     ridgeModel.fit(x_train, y_train)
     scores = cross_val_score(ridgeModel, X, y, cv = 10)
     avg_cross_val_score = np.mean(scores)
     cross_val_scores_ridge.append(avg_cross_val_score)
     alpha.append(i * 0.25)

In [15]:
# predict sby_need with calls & lasso regression
# Lasso
# Develop Lasso(L1) Regression Model:
# Estimate different values for lamda
lamda = []
cross_val_scores_lasso = []
# Loop to compute the different scores
for i in range(1, 9):
    lassoModel = Lasso(alpha = i * 0.25, tol = 0.0925)
    lassoModel.fit(x_train, y_train)
    scores = cross_val_score(lassoModel, X, y, cv = 10)
    avg_cross_val_score = np.mean(scores)
    cross_val_scores_lasso.append(avg_cross_val_score)
    lamda.append(i * 0.25)

In [16]:
# Loop to print the different scores
for i in range(0, len(alpha)):
    print(str(alpha[i])+' : '+str(cross_val_scores_lasso[i]))

0.25 : -0.2498881851603504
0.5 : -0.24383261133568265
0.75 : -0.23337240362347397
1.0 : -0.22294429171192426
1.25 : -0.21358559866124033
1.5 : -0.20490891125863478
1.75 : -0.19670510308419958
2.0 : -0.18911700911966026


In [17]:
# the best value of lamda for the data is 2
# Build the Lasso Regression model for the best lamda
lassoModelChosen = Lasso(alpha = 0)
lassoModelChosen.fit(x_train, y_train)

<ipython-input-17-ac0542b974d8>:4: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lassoModelChosen.fit(x_train, y_train)
/Users/kata/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/Users/kata/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1233146.4539321652, tolerance: 461.6045939262472
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0)

In [18]:
pred = lassoModelChosen.predict(x_test)
pred = pd.Series(pred)
pred = pred.apply(lambda x: 0 if x < 0 else x)

In [19]:
print('MSE of the improved model is:' , mean_squared_error(y_test, pred))

MSE of the improved model is: 8995.414326160024


In [20]:
lassoModelChosen.coef_

array([12.54231462, -7.95664537, -0.22206149,  0.03894655,  0.44965829,
       -0.30081206,  0.        , -1.00335061,  0.19044622])

In [21]:
# Linear regression besser?

In [22]:
model = LinearRegression()
model.fit(x_train, y_train)
pred = model.predict(x_test)

In [23]:
pred = pd.Series(pred)
pred = pred.apply(lambda x: 0 if x < 0 else x)

In [24]:
print('MSE of the improved model is:' , mean_squared_error(y_test, pred))

MSE of the improved model is: 9825.294220719987


In [25]:
model.coef_

array([-1.50142322e-03, -2.37893749e+02,  9.05428771e-03,  4.05404754e-02,
        2.62393346e-01, -1.50142322e-01,  3.55271368e-14, -8.47654560e+00,
        7.76849833e+00])

In [36]:
# df final
sby_need_new = pd.concat([df['sby_need'].iloc[:922], pred], axis=0).reset_index(drop=True)
df['sby_need_new'] = sby_need_new
df

,date,n_sick,calls,n_duty,n_sby,sby_need,dafted,year,month,day_of_week,week,day_of_month,day_of_year,calls_new,sby_need_new
0,2016-04-01,73,8154.0,1700,90,4.0,0.0,2016,4,4,13,1,92,8154.000,4.000000
1,2016-04-02,64,8526.0,1700,90,70.0,0.0,2016,4,5,13,2,93,8526.000,70.000000
2,2016-04-03,68,8088.0,1700,90,0.0,0.0,2016,4,6,13,3,94,8088.000,0.000000
3,2016-04-04,71,7044.0,1700,90,0.0,0.0,2016,4,0,14,4,95,7044.000,0.000000
4,2016-04-05,63,7236.0,1700,90,0.0,0.0,2016,4,1,14,5,96,7236.000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1147,2019-05-23,86,8544.0,1900,90,0.0,0.0,2019,5,3,21,23,143,8041.158,12.994892
1148,2019-05-24,81,8814.0,1900,90,0.0,0.0,2019,5,4,21,24,144,8197.596,18.637970
1149,2019-05-25,76,9846.0,1900,90,146.0,56.0,2019,5,5,21,25,145,7970.616,8.737100
1150,2019-05-26,83,9882.0,1900,90,160.0,70.0,2019,5,6,21,26,146,7914.120,5.747732


In [37]:
df_compare = df.copy()
df_compare

,date,n_sick,calls,n_duty,n_sby,sby_need,dafted,year,month,day_of_week,week,day_of_month,day_of_year,calls_new,sby_need_new
0,2016-04-01,73,8154.0,1700,90,4.0,0.0,2016,4,4,13,1,92,8154.000,4.000000
1,2016-04-02,64,8526.0,1700,90,70.0,0.0,2016,4,5,13,2,93,8526.000,70.000000
2,2016-04-03,68,8088.0,1700,90,0.0,0.0,2016,4,6,13,3,94,8088.000,0.000000
3,2016-04-04,71,7044.0,1700,90,0.0,0.0,2016,4,0,14,4,95,7044.000,0.000000
4,2016-04-05,63,7236.0,1700,90,0.0,0.0,2016,4,1,14,5,96,7236.000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1147,2019-05-23,86,8544.0,1900,90,0.0,0.0,2019,5,3,21,23,143,8041.158,12.994892
1148,2019-05-24,81,8814.0,1900,90,0.0,0.0,2019,5,4,21,24,144,8197.596,18.637970
1149,2019-05-25,76,9846.0,1900,90,146.0,56.0,2019,5,5,21,25,145,7970.616,8.737100
1150,2019-05-26,83,9882.0,1900,90,160.0,70.0,2019,5,6,21,26,146,7914.120,5.747732


In [28]:
df_compare = df_compare[922:]
df_compare['sby_need_new'] = df_compare['sby_need_new']
df_compare

,date,n_sick,calls,n_duty,n_sby,sby_need,dafted,year,month,day_of_week,week,day_of_month,day_of_year,calls_new,sby_need_new
922,2018-10-10,97,7896.0,1900,90,0.0,0.0,2018,10,2,41,10,283,9013.998,65.990715
923,2018-10-11,103,7302.0,1900,90,0.0,0.0,2018,10,3,41,11,284,8552.604,46.586590
924,2018-10-12,106,7506.0,1900,90,0.0,0.0,2018,10,4,41,12,285,8161.170,30.018677
925,2018-10-13,93,9318.0,1900,90,57.0,0.0,2018,10,5,41,13,286,7741.374,12.300954
926,2018-10-14,99,8124.0,1900,90,0.0,0.0,2018,10,6,41,14,287,7697.796,9.835289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1147,2019-05-23,86,8544.0,1900,90,0.0,0.0,2019,5,3,21,23,143,8041.158,12.994892
1148,2019-05-24,81,8814.0,1900,90,0.0,0.0,2019,5,4,21,24,144,8197.596,18.637970
1149,2019-05-25,76,9846.0,1900,90,146.0,56.0,2019,5,5,21,25,145,7970.616,8.737100
1150,2019-05-26,83,9882.0,1900,90,160.0,70.0,2019,5,6,21,26,146,7914.120,5.747732


In [29]:
diff_actual = df_compare['sby_need'] - df_compare['sby_need_new']

In [30]:
diff_actual

922     -65.990715
923     -46.586590
924     -30.018677
925      44.699046
926      -9.835289
           ...    
1147    -12.994892
1148    -18.637970
1149    137.262900
1150    154.252268
1151    -42.845071
Length: 230, dtype: float64

In [31]:
diff_actual = diff_actual.apply(lambda x: 1 if x > 0 else 0)

In [32]:
diff_actual.sum()

64

In [33]:
diff_baseline = df_compare['sby_need'] - df_compare['n_sby']

In [34]:
diff_baseline = diff_baseline.apply(lambda x: 1 if x > 0 else 0)

In [35]:
diff_baseline.sum()

48